# Get delay response time by substructing time of sent mail from recieving time of mail from same mailchain

In [33]:
import pandas as pd

## Load mails from Excel

### Inbox

In [34]:
inbox_df = pd.read_excel(
    r"", sheet_name="Inbox")

### Sent Items

In [35]:
sent_df = pd.read_excel(
    r"", sheet_name="Sent Items")

### Sort data by RecievedTime and reset indexes

In [36]:
inbox_df = inbox_df.sort_values('ReceivedTime')
sent_df = sent_df.sort_values('ReceivedTime')
inbox_df = inbox_df.reset_index(drop=True)
sent_df = sent_df.reset_index(drop=True)

In [37]:
sent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21186 entries, 0 to 21185
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ReceivedTime  21186 non-null  datetime64[ns]
 1   Recipient     21186 non-null  object        
 2   ID            21186 non-null  object        
 3   Full Name     21186 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 662.2+ KB


### Add new column DelayTime

In [38]:
sent_df["DelayTime"] = ""

In [39]:
sent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21186 entries, 0 to 21185
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ReceivedTime  21186 non-null  datetime64[ns]
 1   Recipient     21186 non-null  object        
 2   ID            21186 non-null  object        
 3   Full Name     21186 non-null  object        
 4   DelayTime     21186 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 827.7+ KB


## Find mails into Inbox and Sent by ID. Compare time in both

In [40]:
list_of_id = tuple(set(inbox_df.ID))


for sample_id in list_of_id:
    inbox_iter = 0
    sent_iter = 0

    inbox_rows_list = inbox_df.loc[inbox_df["ID"] == sample_id].index
    sent_rows_list = sent_df.loc[sent_df["ID"] == sample_id].index

    if len(sent_rows_list) == 0:
        continue

    while True:
        inbox_time = inbox_df.iloc[inbox_rows_list[inbox_iter], 0]
        sent_time = sent_df.iloc[sent_rows_list[sent_iter], 0]

        if sent_time > inbox_time:
            sent_df.loc[(sent_df['ReceivedTime'] == sent_time) & (
                sent_df['ID'] == sample_id), "DelayTime"] = sent_time - inbox_time
            #sent_df.iloc[sent_rows_list[sent_iter],-1]  = sent_time - inbox_time
            inbox_iter += 1

            if len(inbox_rows_list) == inbox_iter:
                break
        sent_iter += 1
        if len(sent_rows_list) == sent_iter:
            break

In [ ]:
sent_df.head(20)

### Check the results by loading mail from one mailchain

In [43]:
sample_id = ""
inbox_df.loc[inbox_df["ID"]==sample_id]

,ReceivedTime,Categories,SenderName,ID,Full Name


In [44]:
sent_df.loc[sent_df["ID"]==sample_id]

,ReceivedTime,Recipient,ID,Full Name,DelayTime


## Save results into Excel file

In [45]:
writer = pd.ExcelWriter(r'')

In [46]:
inbox_df.to_excel(writer, sheet_name='Inbox',index=False)
sent_df.to_excel(writer, sheet_name='Sent Items', index=False)
writer.save()